In [1]:
import numpy as np

##########################
# Helper functions/classes
##########################

class WordPair:
    """
    Class representing a pair of words in our vocabulary, along with the cosine similarity
    of the two words.
    """
    def __init__(self, firstWord, secondWord, similarity):
        """
        Initializes the WordPair given two words (strings) and their similarity (float).
        """
        # Ensure that our pair consists of two distinct words
        assert(firstWord != secondWord)
        self.firstWord = firstWord
        self.secondWord = secondWord
        self.similarity = similarity

    def __repr__(self):
        """
        Define the string representation of a WordPair so that a WordPair instance x
        can be displayed using print(x).
        """
        return "Pair(%s, %s), Similarity: %s"%(self.firstWord, self.secondWord, self.similarity)


def sort_by_similarity(word_pairs):
    """
    Given a list of word pair instances, returns a list of the instances sorted
    in decreasing order of similarity.
    """
    return sorted(word_pairs, key=lambda pair: pair.similarity, reverse=True)

def get_similarity(v1, v2):
    """ Returns the cosine of the angle between vectors v1 and v2. """
    v1_unit = v1 / np.linalg.norm(v1)
    v2_unit = v2 / np.linalg.norm(v2)
    return np.dot(v1_unit, v2_unit)


def load_word_list(path):
    """
    Loads a list of the words from the file at path <path>, removing all
    non-alpha-numeric characters from the file.
    """
    with open(path) as handle:
        # Load a list of whitespace-delimited words from the specified file
        raw_text = handle.read().strip().split()
        # Strip non-alphanumeric characters from each word
        alphanumeric_words = map(lambda word: ''.join(char for char in word if char.isalnum()), raw_text)
        # Filter out words that are now empty (e.g. strings that only contained non-alphanumeric chars)
        alphanumeric_words = filter(lambda word: len(word) > 0, alphanumeric_words)
        # Convert each word to lowercase and return the result
        return list(map(lambda word: word.lower(), alphanumeric_words))

def generate_onehot_dict(word_list):
    """
    Takes a list of the words in a text file, returning a dictionary mapping
    words to their index in a one-hot-encoded representation of the words.
    """
    word_to_index = {}
    i = 0
    for word in word_list:
        if word not in word_to_index:
            word_to_index[word] = i
            i += 1
    return word_to_index

def most_similar_pairs(weight_matrix, word_to_index):
    """
    For each word a in our vocabulary, computes the most similar word b to a, along with the
    cosine similarity of a and b.

    Arguments:
        weight_matrix: The matrix of weights extracted from the hidden layer of a fitted
                       neural network.

        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

    Returns: 
        A list of WordPair instances sorted in decreasing order of similarity,
        one representing each word <vocab_word> and its most similar word.
    """
    word_to_feature_repr = get_word_to_feature_repr(weight_matrix, word_to_index)
    result = []
    for word in word_to_feature_repr:
        result.append(most_similar_word(word_to_feature_repr, word))
    return sort_by_similarity(result)

def most_similar_word(word_to_feature_repr, input_word):
    """
    Given a dictionary mapping words to their feature representations (word_to_feature_repr),
    returns the a WordPair instance corresponding to the word
    whose feature vector is most similar to the feature representation of the
    passed-in word (input_word).
    """
    best_word = None
    best_similarity = 0
    input_vec = word_to_feature_repr[input_word]
    for word, feature_vec in word_to_feature_repr.items():
        similarity = get_similarity(input_vec, feature_vec)
        if similarity > best_similarity and np.linalg.norm(feature_vec - input_vec) != 0:
            best_similarity = similarity
            best_word = word
    return WordPair(input_word, best_word, best_similarity)

def get_word_to_feature_repr(weight_matrix, word_to_index):
    """
    Returns a dictionary mapping each word in our vocabulary to its one-hot-encoded
    feature representation.

    Arguments:
        weight_matrix: The matrix of weights extracted from the hidden layer of a fitted
                       neural network.

        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.
    """
    assert(weight_matrix is not None)
    word_to_feature_repr = {}
    for word, one_hot_idx in word_to_index.items():
        word_to_feature_repr[word] = weight_matrix[one_hot_idx]
    return word_to_feature_repr



In [1]:
from keras.models import Sequential
from P3CHelpers import *
import sys

def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          String containing word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation    

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window.
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors representing individual words) and 
                              their corresponding labels (also one-hot-encoded vectors representing words).

                              For each index i, trainX[i] should correspond to a word in
                              <word_list>, and trainY[i] should correspond to one of the words within
                              a window of size <window_size> of trainX[i].
    """
    numWords = len(word_to_index)
    trainX = []
    trainY = []
    allZeroes = [0 for i in range(numWords)]
    for i in range(len(word_list)):
        for j in range(-window_size, window_size + 1):
            if i + j >= 0 and i + j < len(word_list) and j != 0:
                trainXVector = get_word_repr(word_to_index, word_list[i])
                trainX.append(trainXVector)
                trainYVector = get_word_repr(word_to_index, word_list[i+j])
                trainY.append(trainYVector)
                
    return (np.array(trainX), np.array(trainY))

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
from google.colab import files
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
uploaded = files.upload()
filename = "dr_seuss.txt"
# Load in a list of words from the specified file; remove non-alphanumeric characters
# and make all chars lowercase.
sample_text = load_word_list(filename)

# Create dictionary mapping unique words to their one-hot-encoded index
word_to_index = generate_onehot_dict(sample_text)
# Create training data using default window size
trainX, trainY = generate_traindata(sample_text, word_to_index)
# TODO: 1) Create and train model in Keras.      
# vocab_size = number of unique words in our text file. Will be useful when adding layers
# to your neural network
vocab_size = len(word_to_index)
model = Sequential()
model.add(Dense(10, input_dim=(vocab_size)))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
                  metrics=['accuracy'])    
fit = model.fit(trainX, trainY)
# TODO: 2) Extract weights for hidden layer, set <weights> variable below
print("Layer 0 shape" + str(model.layers[0].get_weights()[0].shape))
print("Layer 1 shape" + str(model.layers[1].get_weights()[0].shape))
weights = model.layers[0].get_weights()[0]

# Find and print most similar pairs
similar_pairs = most_similar_pairs(weights, word_to_index)
for pair in similar_pairs[:30]:
    print(pair)

AssertionError: 